In [1]:
import os
import random
import numpy as np
import torch
import config

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-02-26 03:57:12 - get_logger - Writing log to logs/graphnet_20230226-035712.log


In [2]:
def set_gpu_environ():
    """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
    Meant to be used in notebooks only.
    """
    import os
    import subprocess
    query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
    utilization = [int(x.replace(" MiB", "")) for x in query]
    free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
    set_visible = ",".join([str(i) for i in free])
    os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
    print(set_visible)
set_gpu_environ()

1


In [3]:
def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=cfg.WARM_UP_PCT * cfg.EPOCHS,
        num_training_steps=cfg.EPOCHS,
    )



    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )

In [4]:
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

In [ ]:
main('BASELINE_HF_V13_CLS')

Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_24_CLS', 'LOSS_FUNC': <class 'graphnet.training.loss_functions.VonMisesFisher3DLoss'>, 'NUM_WORKERS': 22, 'MODEL_NAME': <class 'icecube.models.EncoderWithDirectionReconstructionV5'>, 'DEVICE': 'cuda:1', 'EPOCHS': 24, 'WD': 0.05, 'BATCH_SIZE': 1280, 'TRN_BATCH_RANGE': [[1, 50], [50, 100], [100, 150], [150, 200], [200, 250], [250, 300], [300, 350], [350, 400], [400, 450], [450, 500], [500, 550], [500, 600]], '__doc__': None}


wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [1, 50]


In [ ]:
k

In [ ]:
k

In [ ]:
km